In [349]:
import csv
import numpy as np
import plotly
import plotly.tools as tls
from plotly.graph_objs import *
import os
import re

def get_csv_data(filepath):
    all_stats = {}
    apps = []
    data = {}
    any_data = False
    with open(filepath, 'r') as data_file:
        reader = csv.reader(data_file)        # define reader object
        state = "start"
        for row in reader:                    # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = [item.upper() for item in row[1:]]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        all_stats[current_stat] = apps,data
                    apps = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                temp = []
                for x in row[1:]:
                    try:
                        temp.append(float(x))
                        any_data = True
                    except ValueError:
                        temp.append(0)
                data[row[0]] = np.array(temp)

    return all_stats


In [ ]:
stats_map = {
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)': 'L1 Total Reads',
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)': 'L1 Read Hits',
    '\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)': 'L1 Read Miss',

    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)' : 'L1 Total TEX',
    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[HIT\]\s*=\s*(.*)' : 'L1 TEX Hits',
    '\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[HIT_RESERVED\]\s*=\s*(.*)' : 'L1 TEX Hits Resvered',

    '\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)' : 'L2 Total Reads',
    '\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)' : 'L2 Read Hits',

    'gpu_sim_cycle\s*=\s*(.*)' : 'Total Cycles',
    # 'L2_BW\s*=\s*(.*)GB\/Sec' : 'L2 BW'
}


this_directory = os.path.abspath(".")
concurrent = get_csv_data("../../per.kernel.stats.csv")
# delete_start = 14
# delete_end = 48
delete_start = 1<<31
delete_end = 1<<31

uids = []
for stat,value in concurrent.items():
    print(stat)
    if not "uid" in stat:
        continue
    apps, data = value
    uids = data["ORIN-SASS"]
uids = np.array(uids).astype(int)
uids = np.delete(uids,np.s_[delete_start:delete_end]).tolist()
uids = [str(x) for x in uids]

colors= ['#0F8C79','#BD2D28','#E3BA22','#E6842A','#137B80','#8E6C8A','#9A3E25', '#E6842A']
stat_count = 0
for stat,value in concurrent.items():
    if stat not in stats_map:
        continue
    stat = stats_map[stat]
    traces = []
    traces_abs = []
    cfg_count = 0
    apps, data = value
    base = np.array(data["ORIN-SASS"])
    base = np.delete(base,np.s_[delete_start:delete_end])
    print(data["ORIN-SASS"])
    for k,v in data.items():
        print(cfg_count)
        
        traces_abs.append(Bar(
            x= uids,
            y= v,
            name= k,
            marker=Marker(color=colors[cfg_count % len(colors)]),
            xaxis='x1',
            )
        )

        if k != "ORIN-SASS":
            v = np.array(v)
            v = np.delete(v,np.s_[delete_start:delete_end])
            to_plot = np.round(v / base,2)
            traces.append(Bar(
                x= uids,
                y= to_plot,
                name=k,
                marker=Marker(color=colors[cfg_count % len(colors)]),
                xaxis='x1', 
                text = to_plot
                )
            )
        cfg_count += 1

    data = Data(traces)
    data_abs = Data(traces_abs)
    layout = Layout(
        title=stat,
        barmode='group',
        bargroupgap=0,
        bargap=0.25,
        showlegend=True,
        yaxis=YAxis(
            title=stat,
        )
    )
    fig = Figure(data=data, layout=layout)
    fig.update_layout(yaxis_range=[0,3])
    fig.update_traces(textangle=0, cliponaxis=True)
    figure_name = re.sub('[^0-9a-zA-Z]+','_',stat)
    print("plotting: " + figure_name)
    outdir = (os.path.join(this_directory,"htmls"))
    if not os.path.exists( outdir ):
        os.makedirs(outdir)
    fig.show()
    # plotly.offline.plot(fig, filename=os.path.join(outdir,figure_name + ".html"),auto_open=False)

    fig = Figure(data=data_abs, layout=layout)
    fig.update_layout(yaxis_range=[0,3])
    fig.update_traces(textangle=0, cliponaxis=True)
    figure_name = re.sub('[^0-9a-zA-Z]+','_',stat)
    print("plotting: " + figure_name)
    outdir = (os.path.join(this_directory,"htmls"))
    if not os.path.exists( outdir ):
        os.makedirs(outdir)
    fig.show()
    # plotly.offline.plot(fig, filename=os.path.join(outdir,figure_name + ".html"),auto_open=False)

In [4]:
apps = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k"]

# addons = '-perf_l2'
addons = '-VISUAL'

configs = ''
configs += "ORIN-SASS" + addons + ','
configs += "ORIN-SASS-concurrent" + addons + ','
# for i in range(12,16):
    # configs += "ORIN-SASS-concurrent-" + str(i) +'GR' + addons + ','
# for i in range(12,16):
    # configs += "ORIN-SASS-concurrent-" + str(i) +'GR-MIG' + addons + ','
for i in range(1,14):
    configs += "ORIN-SASS-concurrent-" + str(i) +'GR-finegrain' +  addons + ','


configs = configs[:-1]
print(configs)



ORIN-SASS-VISUAL,ORIN-SASS-concurrent-VISUAL,ORIN-SASS-concurrent-1GR-finegrain-VISUAL,ORIN-SASS-concurrent-2GR-finegrain-VISUAL,ORIN-SASS-concurrent-3GR-finegrain-VISUAL,ORIN-SASS-concurrent-4GR-finegrain-VISUAL,ORIN-SASS-concurrent-5GR-finegrain-VISUAL,ORIN-SASS-concurrent-6GR-finegrain-VISUAL,ORIN-SASS-concurrent-7GR-finegrain-VISUAL,ORIN-SASS-concurrent-8GR-finegrain-VISUAL,ORIN-SASS-concurrent-9GR-finegrain-VISUAL,ORIN-SASS-concurrent-10GR-finegrain-VISUAL,ORIN-SASS-concurrent-11GR-finegrain-VISUAL,ORIN-SASS-concurrent-12GR-finegrain-VISUAL,ORIN-SASS-concurrent-13GR-finegrain-VISUAL


In [41]:
import os
from datetime import datetime
# save current working directory
os.chdir("/home/tgrogers-raid/a/pan251/accel-sim-framework/util/graphics")
cwd = os.getcwd()
# change cwd to accel-sim-framework
root = os.path.join(cwd,"../../")
os.chdir(root)

today = datetime.now()
# d = today.strftime("%Y%m%d-%H%M")
# open run.sh
run_sh = open(os.path.join(root,"run.sh"), 'w')
run_sh.write("#!/bin/bash\n")
run_sh.write("source ./gpu-simulator/setup_environment.sh\n")
run_sh.write("time=$(date +%Y%m%d-%H%M)\n")

for app in apps:
    cmd = "./util/job_launching/run_simulations.py -B vulkan:" + app + " -C " + configs + " -T ./hw_run/traces/vulkan/ -N run-$time" + "-" + app + "\n"
    run_sh.write(cmd)

# run_sh.write("./util/job_launching/run_simulations.py -B vulkan,compute -C ORIN-SASS" + addons + " -T ./hw_run/traces/vulkan/ -N run-$time\n")

run_sh.write("echo run-$time\n")
run_sh.close()
os.chdir(cwd)

In [9]:
import os

name = "run-20230425-1421"
cmd = "./util/job_launching/get_stats.py -k -R -N " + name + " | tee per.kernel.stats.csv"

print(cmd)
# os.system(cmd)


./util/job_launching/get_stats.py -k -R -N run-20230425-1421 | tee per.kernel.stats.csv


In [3]:
import pandas as pd
import numpy as np
import os
# open per.kernel.stats.csv as dataframe
dir = os.getenv("HOME") + "/accel-sim-framework/"
# csv = "per.kernel.stats.csv"
csv = "per.kernel.stats.csv"
g_count = 48

### Loop the data lines
with open(dir + csv, 'r') as temp_f:
    # get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

temp_f.close()
### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]
    
file = pd.read_csv(dir + csv, header=None, delimiter=",", names=column_names)
file = file.fillna(0)
stat_to_row = {}
for rowi in range(0,file.shape[0]):
    # get the row
    row = file.iloc[rowi]
    if ('---------' in row[0]):
        print(file.iloc[rowi+1][0])
        stat_to_row[file.iloc[rowi+1][0]] = rowi+1

Accel-Sim-build
GPGPU-Sim-build
gpu_tot_sim_insn\s*=\s*(.*)
gpu_tot_sim_cycle\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[GLOBAL_ACC_W\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)
\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_W\]\[HIT\]\s*=\s*(.*)
\s+Total_core_cach

In [8]:
# get cycle rows
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[TEXTURE_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MSHR_HIT\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+Total_core_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
rowi = stat_to_row['gpu_sim_cycle\s*=\s*(.*)']

diff = stat_to_row['GPGPU-Sim-build'] - stat_to_row["Accel-Sim-build"]
# print(diff)
kernel_names = ["NO_ARGS--MESA", "corners--MESA", "fisheye--MESA", "refined--MESA", "klt_tracker--MESA", "all--MESA"]

configs_row = file.iloc[rowi+1]
# new dataframe
all_g = pd.DataFrame()
ga = pd.DataFrame()
ca = pd.DataFrame()
ga["CFG"] = file.iloc[rowi+1:rowi+diff-1,0]
ca["CFG"] = file.iloc[rowi+1:rowi+diff-1,0]
all_c = pd.DataFrame()
for app in apps:
    for kname in kernel_names:
        g = pd.DataFrame()
        c = pd.DataFrame()
        g["CFG"] = file.iloc[rowi+1:rowi+diff-1,0]
        c["CFG"] = file.iloc[rowi+1:rowi+diff-1,0]
        for index, column in configs_row.iteritems():
            if app in column:
                if kname in column:
                    g[column] = file.iloc[rowi+1:rowi+diff-1,index]
                    ga[column] = file.iloc[rowi+1:rowi+diff-1,index]
                elif  "MESA" not in column:
                    c[column] = file.iloc[rowi+1:rowi+diff-1,index]
                    ca[column] = file.iloc[rowi+1:rowi+diff-1,index]
        g.columns = g.iloc[0]
        c.columns = c.iloc[0]
        name = app + "/" + kname.replace('--MESA','')
        if (g.shape[1] > 1):
            g = g.iloc[1:]
            g.set_index("CFG", inplace=True)
            g["SUM"] = g.iloc[:,0:g_count].astype(int).sum(axis=1)
            g = g.iloc[:,-1]
            all_g[name] = g
        # else:
            # print("WARNING - " + name + " has no graphics kernel")
        if (c.shape[1] > 1):
            c = c.iloc[1:]
            c.set_index("CFG", inplace=True)
            c["SUM"] = c.astype(int).sum(axis=1)
            c = c.iloc[:,-1]
            all_c[name] = c
        # else:
            # print("WARNING - " + name + " has no compute kernel")
# sort everything
all_g_to_csv = all_g.iloc[0:2]
all_c_to_csv = all_c.iloc[0:2]
all_c.to_csv("output_c.csv")
all_g.to_csv("output_g.csv")
# # get all MPS rows
# for index,row in all_g.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'GR' in index and 'MIG' not in index and 'finegrain' not in index:
#         all_g_to_csv = all_g_to_csv.append(row)
# # get all MIG rows
# for index,row in all_g.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'MIG' in index:
#         all_g_to_csv = all_g_to_csv.append(row)
# # get all finegrain rows
# for index,row in all_g.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'finegrain' in index:
#         all_g_to_csv = all_g_to_csv.append(row)
# # repeat for computes
# for index,row in all_c.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'GR' in index and 'MIG' not in index and 'finegrain' not in index:
#         all_c_to_csv = all_c_to_csv.append(row)
# # get all MIG rows
# for index,row in all_c.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'MIG' in index:
#         all_c_to_csv = all_c_to_csv.append(row)
# # get all finegrain rows
# for index,row in all_c.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'finegrain' in index:
#         all_c_to_csv = all_c_to_csv.append(row)
# all_c_to_csv.to_csv("output_c.csv")
# all_g_to_csv.to_csv("output_g.csv")
# all_c
ca = ca.iloc[1:]
ga = ga.iloc[1:]
ca.set_index("CFG", inplace=True)
ga.set_index("CFG", inplace=True)
all_ga_to_csv = ga.iloc[0:2]
all_ca_to_csv = ca.iloc[0:2]
ga.to_csv("output_ga.csv")
ca.to_csv("output_ca.csv")
# # repeat
# for index,row in ga.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'GR' in index and 'MIG' not in index and 'finegrain' not in index:
#         all_ga_to_csv = all_ga_to_csv.append(row)
# for index,row in ga.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'MIG' in index:
#         all_ga_to_csv = all_ga_to_csv.append(row)
# for index,row in ga.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'finegrain' in index:
#         all_ga_to_csv = all_ga_to_csv.append(row)

# for index,row in ca.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'GR' in index and 'MIG' not in index and 'finegrain' not in index:
#         all_ca_to_csv = all_ca_to_csv.append(row)
# for index,row in ca.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'MIG' in index:
#         all_ca_to_csv = all_ca_to_csv.append(row)
# for index,row in ca.iterrows():
#     if index == 'ORIN-SASS' or index == 'ORIN-SASS-concurrent':
#         continue
#     if 'finegrain' in index:
#         all_ca_to_csv = all_ca_to_csv.append(row)
# all_ca_to_csv.to_csv("output_ca.csv")
# all_ga_to_csv.to_csv("output_ga.csv")

In [6]:
# get cycle rows
rowi = stat_to_row['gpu_sim_cycle\s*=\s*(.*)']

diff = stat_to_row['GPGPU-Sim-build'] - stat_to_row["Accel-Sim-build"]
# print(diff)
kernel_names = ["NO_ARGS--MESA", "corners--MESA", "fisheye--MESA", "refined--MESA", "klt_tracker--MESA", "all--MESA"]

configs_row = file.iloc[rowi+1]
# new dataframe
all_g = pd.DataFrame()
ga = pd.DataFrame()
ca = pd.DataFrame()
ga["CFG"] = file.iloc[:,0]
ca["CFG"] = file.iloc[:,0]
all_c = pd.DataFrame()
for app in apps:
    for kname in kernel_names:
        for index, column in configs_row.iteritems():
            if app in column:
                if kname in column:
                    ga[column] = file.iloc[:,index]
                elif  "MESA" not in column:
                    ca[column] = file.iloc[:,index]
        name = app + "/" + kname.replace('--MESA','')
# all_c
ca = ca.iloc[1:]
ga = ga.iloc[1:]
ca.set_index("CFG", inplace=True)
ga.set_index("CFG", inplace=True)
ga.to_csv("output_ga.csv")
ca.to_csv("output_ca.csv")

# get cycle rows
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[TOTAL_ACCESS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[MISS\]\s*=\s*(.*)']
# rowi = stat_to_row['\s+L2_cache_stats_breakdown\[GLOBAL_ACC_R\]\[HIT\]\s*=\s*(.*)']
rowi = stat_to_row['gpu_sim_cycle\s*=\s*(.*)']
diff = stat_to_row['GPGPU-Sim-build'] - stat_to_row["Accel-Sim-build"]
# print(diff)
kernel_names = ["NO_ARGS--MESA", "corners--MESA", "fisheye--MESA", "refined--MESA", "klt_tracker--MESA", "all--MESA"]

configs_row = file.iloc[rowi+1]
# new dataframe
all = pd.DataFrame()

for kname in kernel_names:
    start = -1
    count = 0
    for index, column in configs_row.iteritems():
        if (kname in column):
            if (start == -1):
                start = index
            count += 1
    cycles = file.iloc[rowi+1:rowi+diff-1,np.r_[0,start:start+count]]
    cycles.columns = cycles.iloc[0]
    cycles.set_index("CFG", inplace=True)
    cycles = cycles.iloc[1:]
    cycles["SUM"] = cycles.astype(int).sum(axis=1)
    cycles = cycles.iloc[:,-1]
    all[kname.replace('--MESA','')] = cycles

all.to_csv("output.csv")
all